In [3]:
import requests
import os
from tqdm import tqdm
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin, urlparse


#Проверяем, является ли url действительным URL
def is_valid(url):
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)


#Возвращает все URL‑адреса изображений по одному `url`
def get_all_images(url):
    soup = bs(requests.get(url).content, "html.parser")
    urls = []
    for img in tqdm(soup.find_all("img"), "Получено изображение"):
        img_url = img.attrs.get("src")
        if not img_url:
            # если img не содержит атрибута src, просто пропускаем
            continue
        # сделаем URL абсолютным, присоединив имя домена к только что извлеченному URL
        img_url = urljoin(url, img_url)
        # удалим URL‑адреса типа '/hsts-pixel.gif?c=3.2.5'
        try:
            pos = img_url.index("?")
            img_url = img_url[:pos]
        except ValueError:
            pass
        # наконец, если URL действителен
        if is_valid(img_url):
            urls.append(img_url)
    return urls


#Загружает файл по URL‑адресу и помещает его в папку `pathname`
def download(url, pathname):
    # если путь не существует, создать dir
    if not os.path.isdir(pathname):
        os.makedirs(pathname)
    # загружаем тело ответа по частям, а не сразу
    response = requests.get(url, stream=True)

    # получить общий размер файла
    file_size = int(response.headers.get("Content-Length", 0))

    # получаем имя файла и расширение
    filename = os.path.join(pathname, url.split("/")[-1])
    ext = os.path.splitext(filename)[1]
    if ext.lower() not in ['.png', '.jpg']:
        ext = '.jpg' # по умолчанию сохраняем в формате JPG
    filename = os.path.splitext(filename)[0] + ext

    # индикатор выполнения, изменение единицы измерения на байты вместо итераций (по умолчанию tqdm)
    progress = tqdm(response.iter_content(1024), f"Загружен {filename}", total=file_size, unit="B", unit_scale=True, unit_divisor=1024)
    with open(filename, "wb") as f:
        for data in progress.iterable:
            # записываем прочитанные данные в файл
            f.write(data)
            # обновление индикатора выполнения вручную
            progress.update(len(data))


def main(url, path):
    # получить все изображения
    imgs = get_all_images(url)
    for img in imgs:
        # скачать для каждого img
        download(img, path)

In [4]:
main("https://unsplash.com/t/animals", "C:/Users/Пользователь/Desktop/Изображения")

Получено изображение: 100%|██████████| 160/160 [00:00<00:00, 40067.39it/s]
Загружен C:/Users/Пользователь/Desktop/Изображения\p.jpg: 100%|██████████| 43.0/43.0 [00:00<00:00, 8.59kB/s]
Загружен C:/Users/Пользователь/Desktop/Изображения\photo-1526226244945-39c5d1d9e61a.jpg: 100%|██████████| 1.42M/1.42M [00:02<00:00, 630kB/s]
Загружен C:/Users/Пользователь/Desktop/Изображения\profile-1601652880610-dadfc0b29b37image.jpg: 100%|██████████| 176k/176k [00:00<00:00, 348kB/s]
Загружен C:/Users/Пользователь/Desktop/Изображения\photo-1601625463687-25541fb72f62.jpg: 100%|██████████| 5.72M/5.72M [00:07<00:00, 772kB/s]
Загружен C:/Users/Пользователь/Desktop/Изображения\profile-1656504485539-f5f962259353image.jpg: 100%|██████████| 290k/290k [00:00<00:00, 704kB/s]
Загружен C:/Users/Пользователь/Desktop/Изображения\premium_photo-1669725687151-009d3409384c.jpg: 100%|██████████| 3.02M/3.02M [00:02<00:00, 1.19MB/s]
Загружен C:/Users/Пользователь/Desktop/Изображения\profile-1516997253075-2a25da8007e7.jpg: 1

KeyboardInterrupt: 

In [5]:
import zipfile, os, re
 
path = r'C:/Users/Пользователь/Desktop/Изображения/'
pattern = r''
path_to_archive = r'test.zip'
all_files = []
 
with zipfile.ZipFile(path_to_archive, mode='w', compression=zipfile.ZIP_DEFLATED) as zf:
    for root_path, dirs, files in os.walk(path):
        for name in files:
            full_path = os.path.join(root_path, name)
            if os.path.isfile(full_path) and re.match(pattern, name):
                if root_path == path:
                    zf.write(full_path, name)
                else:
                    res = os.path.split(root_path)[-1]
                    zf.write(full_path, os.path.join(res, name))
                all_files.append(full_path)
 
print(os.path.abspath(path_to_archive))
print('\n'.join(all_files))

c:\Users\Пользователь\Desktop\Анализ веб данных\test.zip
C:/Users/Пользователь/Desktop/Изображения/p.jpg
C:/Users/Пользователь/Desktop/Изображения/photo-1520367962648-3110b161a118.jpg
C:/Users/Пользователь/Desktop/Изображения/photo-1520552159191-e28a1d9f0d7e.jpg
C:/Users/Пользователь/Desktop/Изображения/photo-1526226244945-39c5d1d9e61a.jpg
C:/Users/Пользователь/Desktop/Изображения/photo-1601625463687-25541fb72f62.jpg
C:/Users/Пользователь/Desktop/Изображения/photo-1619683060859-26de1af7a0ef.jpg
C:/Users/Пользователь/Desktop/Изображения/photo-1625615188938-ceb8c7f0846a.jpg
C:/Users/Пользователь/Desktop/Изображения/photo-1633315996903-39ee5b9fa6e0.jpg
C:/Users/Пользователь/Desktop/Изображения/premium_photo-1664302673689-6cf27cf23406.jpg
C:/Users/Пользователь/Desktop/Изображения/premium_photo-1669725687132-3e8980ba5002.jpg
C:/Users/Пользователь/Desktop/Изображения/premium_photo-1669725687151-009d3409384c.jpg
C:/Users/Пользователь/Desktop/Изображения/premium_photo-1669725687163-78b8cb62482